# Select Projections

Learn how to project Firestore documents to a subset of fields using FireProx's ``select()`` helper.

## Imports
We will use the built-in testing harness to seed data inside the Firestore emulator.

In [ ]:
from fire_prox import FireProx, AsyncFireProx
from fire_prox.testing import testing_client, async_testing_client, firestore_harness
import asyncio


## Synchronous projections
Create a small dataset and request only the fields you care about.

In [ ]:
with firestore_harness():
    db = FireProx(testing_client())
    users = db.collection('demo_select_users')
    sample = [
        {'name': 'Ada Lovelace', 'country': 'England', 'role': 'Mathematician'},
        {'name': 'Alan Turing', 'country': 'England', 'role': 'Computer Scientist'},
        {'name': 'Grace Hopper', 'country': 'USA', 'role': 'Rear Admiral'},
    ]
    for idx, data in enumerate(sample, start=1):
        doc = users.new()
        for key, value in data.items():
            setattr(doc, key, value)
        doc.save(doc_id=f'user{idx}')
    projection = users.select('name', 'country').get()
projection


You can also stream projections to keep memory usage low.

In [ ]:
with firestore_harness():
    db = FireProx(testing_client())
    users = db.collection('demo_select_users_stream')
    for label in ('Ada', 'Alan', 'Grace'):
        doc = users.new()
        doc.name = label
        doc.country = 'England' if label != 'Grace' else 'USA'
        doc.save(doc_id=label.lower())
    stream_projection = list(users.select('name').stream())
stream_projection


## Async projections with reference hydration
Projected document references automatically hydrate into ``AsyncFireObject`` instances.

In [ ]:
async def async_projection_demo():
    db = AsyncFireProx(async_testing_client())
    users = db.collection('demo_select_users_async')
    mentor = users.new()
    mentor.name = 'Ada Lovelace'
    mentor.country = 'England'
    await mentor.save(doc_id='ada')
    student = users.new()
    student.name = 'Async Student'
    student.mentor = users.doc('ada')
    await student.save(doc_id='student')
    return await (
        users
        .where('name', '==', 'Async Student')
        .select('name', 'mentor')
        .get()
    )

with firestore_harness():
    async_results = asyncio.run(async_projection_demo())
async_results
